In [3]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
user_ratings = pd.read_csv("trial_dataset.csv", index_col=0)
user_ratings = user_ratings.fillna(0)
user_ratings

,action1,action2,action3,romantic1,romantic2,romantic3
user 1,4.0,5.0,3.0,0.0,2.0,1.0
user 2,5.0,3.0,3.0,2.0,2.0,0.0
user 3,1.0,0.0,0.0,4.0,5.0,4.0
user 4,0.0,2.0,1.0,4.0,0.0,3.0
user 5,1.0,0.0,2.0,3.0,3.0,4.0


In [ ]:
def standardise(row_rating):
    new_row_rating = (row_rating - row_rating.mean())/(row_rating.max() - row_rating.min())
    return new_row_rating

std_ratings = user_ratings.apply(standardise)
std_ratings

,action1,action2,action3,romantic1,romantic2,romantic3
user 1,0.36,0.6,0.400000,-0.65,-0.08,-0.35
user 2,0.56,0.2,0.400000,-0.15,-0.08,-0.60
user 3,-0.24,-0.4,-0.600000,0.35,0.52,0.40
user 4,-0.44,0.0,-0.266667,0.35,-0.48,0.15
user 5,-0.24,-0.4,0.066667,0.10,0.12,0.40


In [ ]:
# Item to Item collaborative filtering
similarity_index_for_item = cosine_similarity(std_ratings.T) #Transpose to get similarities for items in rows
print(similarity_index_for_item)
# User to User collaborative filtering
similarity_index_for_user = cosine_similarity(std_ratings)
print(similarity_index_for_user)


[[ 1.          0.70668875  0.81368151 -0.79941088 -0.02539184 -0.91410609]
 [ 0.70668875  1.          0.72310153 -0.84515425 -0.5189993  -0.84337386]
 [ 0.81368151  0.72310153  1.         -0.84794611 -0.3799803  -0.80218063]
 [-0.79941088 -0.84515425 -0.84794611  1.          0.14803913  0.72374686]
 [-0.02539184 -0.5189993  -0.3799803   0.14803913  1.          0.39393939]
 [-0.91410609 -0.84337386 -0.80218063  0.72374686  0.39393939  1.        ]]
[[ 1.          0.76409098 -0.83718317 -0.57786068 -0.7361606 ]
 [ 0.76409098  1.         -0.78062564 -0.60148288 -0.74685173]
 [-0.83718317 -0.78062564  1.          0.23334449  0.64173153]
 [-0.57786068 -0.60148288  0.23334449  1.          0.24550092]
 [-0.7361606  -0.74685173  0.64173153  0.24550092  1.        ]]


In [ ]:
# Item to Item collaborative filtering
items_df = pd.DataFrame(similarity_index_for_item, index=user_ratings.columns, columns=user_ratings.columns)
items_df

,action1,action2,action3,romantic1,romantic2,romantic3
action1,1.000000,0.706689,0.813682,-0.799411,-0.025392,-0.914106
action2,0.706689,1.000000,0.723102,-0.845154,-0.518999,-0.843374
action3,0.813682,0.723102,1.000000,-0.847946,-0.379980,-0.802181
romantic1,-0.799411,-0.845154,-0.847946,1.000000,0.148039,0.723747
romantic2,-0.025392,-0.518999,-0.379980,0.148039,1.000000,0.393939
romantic3,-0.914106,-0.843374,-0.802181,0.723747,0.393939,1.000000


In [ ]:
# User to User collaborative filtering
users_df = pd.DataFrame(similarity_index_for_user, index=user_ratings.T.columns, columns=user_ratings.T.columns)
users_df

,user 1,user 2,user 3,user 4,user 5
user 1,1.000000,0.764091,-0.837183,-0.577861,-0.736161
user 2,0.764091,1.000000,-0.780626,-0.601483,-0.746852
user 3,-0.837183,-0.780626,1.000000,0.233344,0.641732
user 4,-0.577861,-0.601483,0.233344,1.000000,0.245501
user 5,-0.736161,-0.746852,0.641732,0.245501,1.000000


In [ ]:
# Item to Item collaborative filtering
def get_similar_movies_based_on_items(movie, user_rating):
    similarity = items_df[movie]*(user_rating-2.5)
    similarity = similarity.sort_values(ascending=False)
    return similarity

print(get_similar_movies_based_on_items("romantic3", 1))

action1      1.371159
action2      1.265061
action3      1.203271
romantic2   -0.590909
romantic1   -1.085620
romantic3   -1.500000
Name: romantic3, dtype: float64


In [ ]:
# User to User collaborative filtering
def get_similar_movies_based_on_users(user, user_rating):
    similarity = users_df[user]*(user_rating-2.5)
    similarity = similarity.sort_values(ascending=False)
    return similarity

print(get_similar_movies_based_on_users("user 1", 1))

user 3    1.255775
user 5    1.104241
user 4    0.866791
user 2   -1.146136
user 1   -1.500000
Name: user 1, dtype: float64


In [ ]:
test_user = [("action1", 5),("romantic2", 1),("romantic3",1)]

similar_movies_based_on_item = pd.DataFrame()

for movie,rating in action_lover:
    similar_movies_based_on_item = similar_movies_based_on_item.append(get_similar_movies_based_on_items(movie,rating), ignore_index=True)

similar_movies_based_on_item.head()
similar_movies_based_on_item.sum().sort_values(ascending=False)


action1      3.909247
action2      3.810282
action3      3.807445
romantic2   -2.154389
romantic1   -3.306206
romantic3   -4.376174
dtype: float64